In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import scipy
import os
import xwidgets

In [2]:
def load_data(img_path, msk_path, describe=True):
    img = np.load(img_path)['data']
    msk = np.load(msk_path)['labels']
    
    # only necessary if input is not already in an (x,y,z) format
    img = img.transpose(1,2,0)
    msk = msk.transpose(1,2,0)
    
    img = img[1:,:,:]
    msk = msk[1:,:,:]
    
    if describe:
        print(f'Image Shape: {img.shape} -- Image Min/Max: {(np.min(img), np.max(img))}')
        print(f'Mask Shape: {msk.shape} -- Mask Min/Max: {(np.min(msk), np.max(msk))}')
    
    return img, msk

# Xrathus data path.
dataset_path = '/home/jovyan/projects/seam/seismic_facies_identification_challenge/data/dataset-14-v1/'

data_path = os.path.join(dataset_path, 'data_train.npz')
labels_path = os.path.join(dataset_path, 'labels_train.npz')
#data_path = "/home/jovyan/data/xrathus/welcome/dataset-13-v2/sampledata/example3dseismicviewer.npy"
#labels_path = "/home/jovyan/data/xrathus/welcome/dataset-13-v2/sampledata/example3dseismicviewer2.npy"
dataset_all,labels_all = load_data(data_path, labels_path, describe = True)
#dataset = dataset_all[100:250,250:400,:]
#labels = labels_all[100:250,250:400,:]

Image Shape: (781, 590, 1006) -- Image Min/Max: (-5195.5234, 5151.7188)
Mask Shape: (781, 590, 1006) -- Mask Min/Max: (1, 6)


In [3]:
def modifyNeighbors(array, x, y, z, d, value=1):
    # make sure you doesn't hit outside array boundaries
    x1 = x2 = y1 = y2 = z1 = z2 = d
    if x-x1 < 0:
        x1 = x
    if x+x2+1 > array.shape[0]:
        x2 = array.shape[0]-1-x
    if y-y1 < 0:
        y1 = y
    if y+y2+1 > array.shape[1]:
        y2 = array.shape[1]-1-y
    if z-z1 < 0:
        z1 = z
    if z+z2+1 > array.shape[2]:
        z2 = array.shape[2]-1-z

    array[x-x1:x+x2+1, y-y1:y+y2+1, z-z1:z+z2+1] = value
    

def generateBinaryBoundaryMask(msk, padding=5):
    # init empty
    conditionalMsk = np.zeros(msk.shape)
    
    # use np.diff to find where transition in values occurs
    for i in range(3):
        tempMsk = np.diff(msk, axis = i)
        tempMsk2 = np.zeros(msk.shape)
        #add last row/column, since diff shape is 1 less than true length
        if i == 0:
            tempMsk2[:-1,:,:] = tempMsk
            tempMsk2[-1,:,:] = tempMsk[-1,:,:]
        elif i == 1:
            tempMsk2[:,:-1,:] = tempMsk
            tempMsk2[:,-1,:] = tempMsk[:,-1,:]
        elif i == 2:
            tempMsk2[:,:,:-1] = tempMsk
            tempMsk2[:,:,-1] = tempMsk[:,:,-1]
        conditionalMsk = ((conditionalMsk!=0) | (tempMsk2!=0)).astype(int)
                
        del tempMsk, tempMsk2
    
    # change neighbors of 1's to include all areas within padding pixels distance
    for x in np.argwhere(conditionalMsk>0):
        modifyNeighbors(conditionalMsk, x[0], x[1], x[2], padding)
        
    return conditionalMsk

In [4]:
newMask = generateBinaryBoundaryMask(labels_all, padding=2)

In [5]:
newMask.shape

(781, 590, 1006)

In [6]:
# save the newly generated dataset
np.savez_compressed("./boundaries",boundaries=newMask)

In [ ]:
# start with this line if you have saved boundaries before

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import scipy
import os
import xwidgets

# Xrathus data path.
dataset_path = '/home/jovyan/projects/seam/seismic_facies_identification_challenge/data/dataset-14-v1/'
data_path = os.path.join(dataset_path, 'data_train.npz')

newMask = np.load("./boundaries_5.npz")['boundaries']
img = np.load(data_path)['data']

img = img.transpose(1,2,0)
img = img[1:,:,:]

print(newMask.shape)
print(img.shape)

(781, 590, 1006)
(781, 590, 1006)


In [10]:
viewer = xwidgets.SeismicViewer3D(dataset_all)
#viewer.addDataSource(dataSource=labels_all)
viewer.addDataSource(dataSource=newMask)
#viewer.setDataSourceMinMax(dataSourceID=2, dataMin=1, dataMax=6)
viewer.setSize(width=600, height=900)
viewer.setAspectRatio(x=1, y=1, z=2)
viewer.setLabels(x='X', y='Y', z='Z')
viewer.setColorscales(colorscales=['Greys', 'RdBu', 'Rainbow'])
viewer.invertAxis(z=True)
viewer.show()